## Node Importance

### Centrality
__Uses__
- Influential nodes in a social network
- Nodes that disseminate information to many nodes or prevent epidemics
- Hubs in a transportation network
- Important pages on the Web
- Nodes that prevent the network from breaking up

__Measures__
- Degree Centrality
    - important nodes have many connections, i.e., number of neighbors
    - Undirected networks: use degree
        - $C_{deg}(v) = \frac{d_{v}}{{|N|-1}}$, Where N is the set of nodes in the network and $d_{v}$ is the degree of node v
    - Directed networksL use in-degree or out-degree
        - $C_{deg}^{in} = \frac{d_{v}^{in}}{{|N|-1}}$ 
        - $C_{deg}^{out} = \frac{d_{v}^{out}}{{|N|-1}}$ 
<br></br>
- Closeness Centrality
    - important nodes are close to other nodes
    - Formula:
        - $C_{close}(v) = \frac{|N|-1}{{\sum_{u\in N!(v)}^{d(v,u)}}}$
        - closeCent = nx.closeness_centrality(G)
    - How to reach the closeness centrality of a node when it cannot reach all other nodes?
        - Option I: Consider only nodes that node L can reach
        - Option II: Consider only nodes that node L can reach, but normalize by the fraction of nodes L can reach
            - $C_{close(L)} = \begin{pmatrix}|\frac{R(L)}{{N-1}}|\end{pmatrix}\frac{|R(L)|}{{\sum_{u\in R(L)}^{d(L,u)}}}$
            - closeCent = nx.closeness_centrality(G,normalized = True)

In [4]:
## Degree Centrality
# Undirected Networks
import networkx as nx
G = nx.karate_club_graph()
G = nx.convert_node_labels_to_integers(G,first_label = 1)
degCent = nx.degree_centrality(G)
degCent[34] # 17/33

0.5151515151515151

#### Directed Networks
- indegCent = nx.in_degree_centrality(G)
- indegCent[1]
- outdegCent = nx.out_degree_centrality(G)

### Betweenness Centrality
__Assumption__
- Important nodes connect other nodes
<br></br>

__Formula__
- $C_{btw}(v) = \sum_{s,t\in N}\frac{\sigma_{s,t}(v)}{{\sigma_{s,t}}}$ 
- sum of shortest paths between nodes $s$ and $t$ that pass through node $v$
- over number of shortest paths between nodes $s$ and $t$   
    
__Endpoint__
- we can either include or exclude node $v$ as node $s$ and $t$ in the computaton of $C_{btw}(v)$

__ Unconnected?__
- e.g., Node $D$ can not be reached by any other node
- Only considering nodes $s,t$ having at least one shortest path inbetween 

__Normalization__
- Betweenness centrality values will be larger in graphs with many nodes. 
- To control for this, we divide centrality values by the number of pairs of nodes in the graph(excluding $v$)
    - $\frac{1}{{2}}(|N|-1)(|N|-2)$ in undirected graphs
    - $(|N|-1)(|N|-2)$ in directed graph


In [6]:
btwnCent = nx.betweenness_centrality(G,normalized = True, endpoints = False)
import operator
sorted(btwnCent.items(),key = operator.itemgetter(1),reverse = True)[0:5]

[(1, 0.4376352813852815),
 (34, 0.30407497594997596),
 (33, 0.14524711399711404),
 (3, 0.14365680615680615),
 (32, 0.13827561327561327)]

__Complexity__
- Computationally expensive
__Approximation__
- using a sample of nodes
    - like stochastic gradient descent

In [9]:
btwnCent_approx = nx.betweenness_centrality(G,normalized=True,endpoints=False,k=10) # using 10 rather than all the nodes
sorted(btwnCent_approx.items(),key = operator.itemgetter(1),reverse=True)[0:5]

[(1, 0.484459626022126),
 (34, 0.25483390452140453),
 (33, 0.18650733525733526),
 (32, 0.10650042087542086),
 (3, 0.10410353535353536)]

__Subsets__
- most important nodes between the source & target node sets
- $s$ *always* from source nodes, $t$ from target nodes

In [13]:
btwnCent_subset = nx.betweenness_centrality_subset(G,[34,33,21,30,16,27,15,23,10],[1,4,13,11,6,12,17,7],normalized=True)
sorted(btwnCent_subset.items(),key = operator.itemgetter(1),reverse=True)[0:5]

[(1, 0.04899515993265994),
 (34, 0.028807419432419434),
 (3, 0.018368205868205867),
 (33, 0.01664712602212602),
 (9, 0.014519450456950456)]

__Edges__
- $C_{btw}(e) = \sum_{s,t\in N}\frac{\sigma_s,t(e)}{{\sigma_{s,t}}}$

In [15]:
btwnCent_edge = nx.edge_betweenness_centrality(G,normalized=True)
sorted(btwnCent_edge.items(),key = operator.itemgetter(1),reverse = True)[0:5]

[((1, 32), 0.12725999490705373),
 ((1, 7), 0.07813428401663695),
 ((1, 6), 0.07813428401663694),
 ((1, 3), 0.07778768072885717),
 ((1, 9), 0.07423959482783016)]

In [18]:
btwnCent_edge_subset = nx.edge_betweenness_centrality_subset(G,[34,33,21,30,16,27,15,23,10],[1,4,13,11,6,12,17,7],normalized=True)
sorted(btwnCent_edge_subset.items(),key = operator.itemgetter(1),reverse=True)[0:5]

[((1, 9), 0.01366536513595337),
 ((1, 32), 0.01366536513595337),
 ((14, 34), 0.012207509266332794),
 ((1, 3), 0.01211343123107829),
 ((1, 6), 0.012032085561497326)]

## Page Rank
- Basic ideas: developed to measure the importance of webpages from the hyperlink network structure
    - assign a score of importance to each node
    - important nodes are those with many in-links from important pages
    - mainly useful for directed networks
- Steps
    - $n =$ number of nodes in the network; $k=$ number of steps
    - Assign all nodes a PageRank of $\frac{1}{{n}}$
    - Perform the __Basic PageRank Update Rule__ $k$ times

### Basic Page Rank
__Basic PageRank Updata Rule__
- Each node gives an equal share of its current PageRank to all the nodes it links to
- Update the PR of the node $A$ by multiply $\frac{C_{A}}{{C_{all}}}$, Where $C$ refers to all the projections out of $C$
- Iterate until convergence, i.e., changing a little

### Scaled PageRank 
__Random walk of k steps__
- Choose a random node; Choose a random edge; To the next node; Repeat $k$ times
- BPR refers to the probability of taking a random walk of $k$ steps and land on the specific node 
<br></br>

__Damping Parameter $\alpha$__

- with probability $\alpha$, choose an outgoing edge at random and follow it to the next node
- with probability $1 - \alpha$, choose a node at random at go to it
- repeat $k$ times
- avoid stucks on certain nodes
- $\alpha \in [0.8,0.9]$ 
- works well for large network
- $nx.pagerank(G,\alpha = .8)$

## Hubs and Authorities

- Root
    - set of highly relevant web pages - potential *authorities*
        - pages containing the query string
- Hubs
    - Find all pages that link to a page in root: potential *hubs*

- Base
    - root nodes and any node that links to a node in root
    - aka. Root + Hub

### HITS Algorithm
- Computing $k$ iterations of HITS algorithm to assign an *authority* score and *hub* score to each node
- Assign each node an authority and hub score of 1
- Apply __Authority Update Rule__: each node's <font color = 'red'> *authority*</font> score is the sum of <font color = 'red'>*hub*</font>  scores of each node that <font color = 'red'>*points to it*</font>
- Apply __Hub Update Rule__: each node's <font color = 'red'> *hub*</font> score is the sum of <font color = 'red'>*authority*</font>  scores of each node that <font color = 'red'>*it points to*</font>
- Normalize Authority and Hub scores: $auth(j) = \frac{auth(j)}{{\sum_{i\in N}auth(i)}}$; same for Hub
- Repeat $k$ times until convergence
- $nx.hits(G)$ with two dictionaries

## Comparing Centrality Measures

In [4]:
import networkx as nx
G = nx.Graph()
G.add_edges_from([('A','B'),
                  ('A','C'),
                  ('B','D'),
                  ('C','D'),
                  ('C','E'),
                  ('D','E'),
                  ('D','G'),
                  ('E','G'),
                  ('G','F')
])
nx.edge_betweenness_centrality(G,normalized = False)
nx.betweenness_centrality(G,normalized = True, endpoints = False)

{'A': 0.03333333333333333,
 'B': 0.07777777777777777,
 'C': 0.18888888888888888,
 'D': 0.38888888888888884,
 'E': 0.1111111111111111,
 'F': 0.0,
 'G': 0.3333333333333333}

In [16]:
G2 = nx.DiGraph()
G2.add_edges_from([('A','B'),
                  ('B','C'),
                  ('A','C'),
                  ('C','A'),
                   ('D','C'),
                  ])
import numpy as np
import pandas as pd
nx.hits(G2,max_iter=2,tol = 1,normalized=True)

({'A': 0.4,
  'B': 0.2857142857142857,
  'C': 0.02857142857142857,
  'D': 0.2857142857142857},
 {'A': 0.06666666666666667,
  'B': 0.26666666666666666,
  'C': 0.6666666666666666,
  'D': 0.0})